In [1]:
!pip install chromadb
!pip install -U langchain-community
!pip install cohere

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import TokenTextSplitter
import pandas as pd
from langchain.llms import Cohere
from langchain.document_loaders import DataFrameLoader
import os
from dotenv import load_dotenv

In [4]:

load_dotenv("app.env")


cohereApi = os.getenv('COHERE_API')


In [5]:
llm = Cohere(
    cohere_api_key=cohereApi,
    model= "command-r-plus",
)


/tmp/ipykernel_165/215183641.py:1: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(


In [6]:
df = pd.read_csv("hf://datasets/Ahmedhany216/Islamic-Books/my_dataframe.csv")

In [7]:
df.head()

,hadith,Reference
0,قال حدثني الليثي عن مالك بن أنس عن ابن شهاب أن...,موطأ الإمام مالك
1,و حدثني يحيى عن مالك عن زيد بن أسلم عن عطاء بن...,موطأ الإمام مالك
2,و حدثني يحيى عن مالك عن يحيى بن سعيد عن عمرة ب...,موطأ الإمام مالك
3,و حدثني عن مالك عن زيد بن أسلم عن عطاء بن يسار...,موطأ الإمام مالك
4,و حدثني عن مالك عن نافع مولى عبد الله بن عمر أ...,موطأ الإمام مالك


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62169 entries, 0 to 62168
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   hadith     62169 non-null  object
 1   Reference  62169 non-null  object
dtypes: object(2)
memory usage: 971.5+ KB


In [9]:
len(df["hadith"])

62169

In [10]:
df.isna().sum()

hadith       0
Reference    0
dtype: int64

In [11]:
df.drop_duplicates(subset=["hadith"],inplace=True)

In [12]:
df["hadith"].duplicated().sum()

0

In [13]:
llmLoader = DataFrameLoader(df,page_content_column="hadith")


In [14]:
llmData = llmLoader.load()


In [15]:
llmData[0].dict()

/tmp/ipykernel_165/3900061053.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  llmData[0].dict()


{'id': None,
 'metadata': {'Reference': 'موطأ الإمام مالك'},
 'page_content': 'قال حدثني الليثي عن مالك بن أنس عن ابن شهاب أن عمر بن عبد العزيز أخر الصلاة يوما فدخل عليه عروة بن الزبير فأخبره أن المغيرة بن شعبة أخر الصلاة يوما وهو بالكوفة فدخل عليه أبو مسعود الأنصاري فقال ما هذا يا مغيرة أليس قد علمت أن جبريل نزل فصلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم قال بهذا أمرت فقال عمر بن عبد العزيز اعلم ما تحدث به يا عروة أو إن جبريل هو الذي أقام لرسول الله صلى الله عليه وسلم وقت الصلاة قال عروة كذلك كان بشير بن أبي مسعود الأنصاري يحدث عن أبيه قال عروة ولقد حدثتني عائشة زوج النبي صلى الله عليه وسلم أن رسول الله صلى الله عليه وسلم كان يصلي العصر والشمس في حجرتها قبل أن تظهر.',
 'type': 'Document'}

In [16]:
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base",  
    chunk_size=800,
    chunk_overlap=100
)

In [17]:
tokens_chunks = text_splitter.create_documents([i.page_content for i in llmData], metadatas=[i.metadata for i in llmData])


In [18]:
model_em = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


/tmp/ipykernel_165/619090216.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_em = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
2025-08-10 15:45:33.177141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754840733.351113     165 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754840733.400539     165 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register fa

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [19]:
from langchain.vectorstores import Chroma 
saveToDir= "/kaggle/working/"
docs_ids=list(range(len(tokens_chunks)))
docs_ids=[str(i) for i in docs_ids]
vector_db = Chroma.from_documents(tokens_chunks,model_em,persist_directory=saveToDir,ids=docs_ids)

In [20]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [21]:
qna_template = "\n".join([
    "استخرج إجابة السؤال التالي بناءً على الحديث المقدم وارجع الحديث نصا مع الاجابه",
    "إذا لم يكن في الحديث ما يساعد على الإجابة، قل: 'لا توجد إجابة الاحاديث'.",
    "### الحديث:",
    "{context}",
    "",
    "### السؤال:",
    "{question}",
    "",
    "### الإجابة:",
])


In [27]:
qnaPrompt = PromptTemplate(
    template=qna_template,
    input_variabels=["context","question"],
    verbose=True
)

In [28]:
question = "الكذاب"
similar_docs = vector_db.similarity_search(question,k=20)

In [29]:
stuffChain = load_qa_chain(llm,chain_type="stuff",prompt=qnaPrompt)

In [30]:
answer = stuffChain({"input_documents":similar_docs,"question":question},return_only_outputs=True)

In [31]:
answer

{'output_text': 'الإجابة: الكذاب هو الشخص الذي يُعرف بكثرة الكذب ويكون الكذب صفة لازمة له، وقد أشارت الأحاديث النبوية إلى عقوبة الكذب على الله ورسوله، وأنه من أكذب الناس، كما بينت أن من علامات الساعة خروج كذابين ودجالين. \n\nالحديث: \nعن عبد الله بن مسعود رضي الله عنه قال: قال رسول الله صلى الله عليه وسلم: "إن الصدق يهدي إلى البر، وإن البر يهدي إلى الجنة، وإن الرجل ليصدق حتى يكتب عند الله صديقا، وإن الكذب يهدي إلى الفجور، وإن الفجور يهدي إلى النار، وإن الرجل ليكذب حتى يكتب عند الله كذابا". صحيح مسلم\n\nوفي حديث آخر، قال رسول الله صلى الله عليه وسلم: "كفى بالمرء إثما أن يحدث بكل ما سمع، كفى بالمرء كذبا أن يحدث بكل ما سمع". صحيح مسلم'}